# Linear Group Contribution CMC Model

This notebook demonstrates feature extraction, linear model fitting, and subsequent feature selection for a group contribution model of CMC prediction.

In [1]:
from collections import defaultdict
from pathlib import Path
from typing import Dict, List, NamedTuple

from rdkit.Chem import MolFromSmiles
from sklearn.feature_selection import SelectFromModel, RFECV
from sklearn.linear_model import ElasticNetCV, RidgeCV, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from camcann.data.io import DataReader, Datasets
from camcann.data.featurise import ECFPCountFeaturiser, SMILESHashes

HERE = Path(".")
HASH_PATH =  HERE / "full_hash.csv"
FEATURES_PATH = HERE / "features_df.csv"
RESULTS_PATH = HERE / "results.csv"
GROUP_CONTRIBS_PATH = HERE / "group_contribs.csv"

USE_SAVED_HASH: bool = False

## Data Loading

First, we load CMC values for 306 surfactants from a precompiled dataset, and convert the SMILES representations of every molecule to an RDKit `Mol` object.

In [2]:
data_reader = DataReader(Datasets.QIN)
all_data = data_reader.df
all_data.describe()

,log CMC
count,202.000000
mean,3.144909
std,1.209451
min,-0.795880
25%,2.397940
50%,3.356955
75%,3.967309
max,5.301030


In [3]:
all_molecules = [MolFromSmiles(smile) for smile in all_data.SMILES]
all_targets = all_data["log CMC"]

## Group counts extraction

Next, we identify all the groups with $\text{radius} \leq 2$ that appear in the molecules. Every group, has a unique hash, which is computed by the extraction algorithm, as well as its own index, $j$, which dictates the column of the resulting feature matrix, $\mathbf{G}$, that contains the count for that group, $N_j$. Each row of $\mathbf{G}$ corresponds to a molecule in the input array, $\vec{m}$:

$$\mathbf{G}_{ij} = N_j(\vec{m}_i).$$

The algorithm for extracting groups and their counts is a variant of ECFP. It iteratively updates a _hash map_, which contains the groups' SMILES fragments, hashes and indexes. It is important to store this; we can only learn group contributions for groups that appear in the training data, so that we must discard groups in the test data that do not appear in the hash map.

In [4]:
if HASH_PATH.exists() and USE_SAVED_HASH:
    smiles_hashes = SMILESHashes.load(HASH_PATH)
    featuriser = ECFPCountFeaturiser(smiles_hashes)
else:
    featuriser = ECFPCountFeaturiser()

In [5]:
all_features = featuriser.featurise_molecules(all_molecules, 2, add_new_hashes=True)
if USE_SAVED_HASH:
    featuriser.smiles_hashes.save(HASH_PATH)
print(f"Number of unique groups: {len(featuriser.smiles_hashes)}.")

Number of unique groups: 482.


The following snippet labels the columns so that we know which SMILES fragment each count is referring to. This isn't strictly necessary for fitting the data, but it is vital for interpretation.

In [6]:
features_df = featuriser.label_features(all_features, all_data.SMILES)
features_df.to_csv(FEATURES_PATH)

In [7]:
count_nonzero = features_df > 0
nnz = count_nonzero.sum()
num_shared = (nnz > 1).sum()
print(f"Number of groups that occur in multiple compounds: {num_shared}")

Number of groups that occur in multiple compounds: 344


Even using the full dataset, about a third of the groups we've discovered only appear once. These aren't useful features; the contributions will only apply to a single molecule, so we're likely to overfit.

## Model fitting and evaluation

To properly evaluate our model, we're going to use 10-fold cross-validation. For each of these folds, we will do the following procedure independently, so that we effectively simulate the real-world scenario of using the model to predict CMCs for arbitrary molecules that we've not seen before:

1. **Initial feature selection**: Remove groups that appear in only a single molecule in the training subset.
2. **Model fitting**: Fit a linear model using the count matrices. The model we're using is described in more detail below; there are some important considerations. One of these is that this model should be able to assign an importance score to each group.
3. **Final feature selection**: Using the group importance scores from the previous step, identify a smaller set of groups that retains similar predictive power to the full set. Having a smaller set of groups is ideal because we want to make the model more generalisable (we want to prioritise more common groups) and as simple as possible.


In [8]:
class FoldResults(NamedTuple):
    """Contains the results for a single fold's evaluation."""
    initial_groups: pd.Series
    pipeline: Pipeline
    test_rmse: float
    
    @property
    def train_cv_rmse(self) -> float:
        """Get the best RMSE from the train set cross-validation."""
        # mses = self.pipeline[-1].cv_values_
        # return np.min(np.mean(mses, axis=1))
        return -self.pipeline[-1].best_score_
    
    @property
    def reduced_num_features(self) -> int:
        """Get the number of features after final selection."""
        return self.pipeline[1].get_support().sum()
    
    def group_weights(self, all_groups: List[str]) -> Dict[str, float]:
        """Get the weights associated with each group after final training."""
        weights = self.pipeline[-1].coef_
        initial_idxs = np.flatnonzero(self.initial_groups.values)

        final_groups = self.pipeline[1].get_support()
        final_idxs = initial_idxs[final_groups]

        return {all_groups[idx]: weight for idx, weight in zip(final_idxs, weights)}

In [9]:
def single_fold_routine(train_idxs, test_idxs, linear: bool = True) -> FoldResults:
    """Perform a single fold training routine."""
    # Get train/test data split
    train_df = features_df.iloc[train_idxs]
    test_df = features_df.iloc[test_idxs]

    train_targets = all_targets.iloc[train_idxs]
    test_targets = all_targets.iloc[test_idxs]

    # 1. Remove features that only occur once
    has_group = train_df > 0
    include_group = has_group.sum() > 1

    train_feats = train_df.iloc[:, include_group.values]
    test_feats = test_df.iloc[:, include_group.values]

    # 2. Train initial model and 3. feature selection
    elastic_net = ElasticNetCV(l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1])
    selector = SelectFromModel(elastic_net, threshold="mean")
    if linear:
        out = RidgeCV(scoring="neg_root_mean_squared_error")
    else:
        out = GridSearchCV(
                MLPRegressor((15, 15), solver="lbfgs", max_iter=2000, random_state=0),
                {"alpha": list(10.0 ** -np.arange(1, 7))},
                scoring="neg_root_mean_squared_error",
            )
    pipe = make_pipeline(
        StandardScaler(),
        selector,
        out
    )
    pipe.fit(train_feats, train_targets)

    test_pred = pipe.predict(test_feats)
    test_rmse = np.sqrt(mean_squared_error(test_targets, test_pred))

    return FoldResults(include_group, pipe, test_rmse)

In [10]:
fold_results: List[FoldResults] = []
for train_idx, test_idx in data_reader.cv_indexes:
    fold_results.append(single_fold_routine(train_idx, test_idx, True))

c:\Users\Alexander\anaconda3\envs\molplotly\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.094e-02, tolerance: 2.089e-02
  model = cd_fast.enet_coordinate_descent(
c:\Users\Alexander\anaconda3\envs\molplotly\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.074e-02, tolerance: 1.905e-02
  model = cd_fast.enet_coordinate_descent(
c:\Users\Alexander\anaconda3\envs\molplotly\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider in

## Results

Here we collect the results for each fold and the weights associated with their top 10 features.

In [11]:
results = defaultdict(list)
group_weights: List[Dict[str, float]] = []
for fold, result in enumerate(fold_results):
    results["Fold"].append(fold)
    results["Train RMSE"].append(result.train_cv_rmse)
    results["Initial num features"].append(result.initial_groups.sum())
    results["Final num features"].append(result.reduced_num_features)
    results["Test RMSE"].append(result.test_rmse)
    
    weights_dict = result.group_weights(featuriser.smiles_hashes.smiles)
    weights_dict["Fold"] = fold
    group_weights.append(weights_dict)

groups_df = pd.DataFrame(group_weights)
groups_df.to_csv(GROUP_CONTRIBS_PATH)

results_df = pd.DataFrame(results)
results_df.to_csv(RESULTS_PATH)

In [12]:
print(results_df)

   Fold  Train RMSE  Initial num features  Final num features  Test RMSE
0     0    0.384374                   337                  76   0.326464
1     1    0.386920                   342                  76   0.388470
2     2    0.351982                   330                  81   0.421738
3     3    0.375577                   319                  75   0.405892
4     4    0.374582                   323                  69   0.498724
5     5    0.395113                   339                  90   0.798534
6     6    0.391357                   344                  79   0.444982
7     7    0.377673                   325                  70   0.437559
8     8    0.386638                   337                  74   0.486744
9     9    0.295568                   340                  53   0.472016


In [13]:
print(groups_df.describe())

           [CH2]  [CH2][CH2][CH2]  [CH2][CH2][CH]  [CH2][CH]([CH2])[OH]  \
count  10.000000        10.000000       10.000000              8.000000   
mean   -0.199515        -0.504738       -0.221200             -0.059062   
std     0.126427         0.046657        0.049357              0.011613   
min    -0.465445        -0.606805       -0.313623             -0.076437   
25%    -0.257691        -0.528702       -0.244842             -0.066020   
50%    -0.186089        -0.497513       -0.214449             -0.055877   
75%    -0.126422        -0.471264       -0.201044             -0.051880   
max     0.005911        -0.446865       -0.140184             -0.044975   

       [CH2][OH]  [CH2][CH2][CH2][CH2][CH2]        [O]  \
count  10.000000                  10.000000  10.000000   
mean   -0.079313                  -0.600491  -0.138656   
std     0.059582                   0.075668   0.045194   
min    -0.185920                  -0.807864  -0.258352   
25%    -0.102683                  

In [14]:
print(f"Total score: {results_df['Test RMSE'].mean()}")

Total score: 0.46811250941211213


In [18]:
groups_df.fillna(0, inplace=True)
avg_contribs = groups_df.mean(axis=0)
print(avg_contribs.sort_values(ascending=False, key=np.abs).head(20))

Fold                         4.500000
[CH2][CH2][CH2][CH2][CH2]   -0.600491
[CH2][CH2][CH2]             -0.504738
[CH2][CH2][CH]              -0.221200
[CH2]                       -0.199515
[O][CH2][CH2][OH]           -0.171524
[Cl-]                        0.157110
[Br-]                        0.155580
[O]                         -0.138656
[C]                         -0.112538
dtype: float64
